In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm, trange
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import numpy as np
import librosa
import torch
import laion_clap
# from extraction.vgg_sound import *
# 在aws notebook环境需要加上这个
import sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, "..")))

from audioldm import image_to_audio, build_model, clap_to_audio
import torch
import torchaudio
from transformers import AutoProcessor, AutoModel
import soundfile as sf 





/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:



class V2AMapperMLP(nn.Module):
    """
    将(64,512)的clip特征先池化到(1,512),
    再映射到(1,512).
    """
    def __init__(self, input_dim=512, hidden_dim=1024, output_dim=512):
        super().__init__()
        # 可以先做一个简单的线性层, 或者堆叠多层
        self.pooling = nn.AdaptiveAvgPool2d((1, input_dim))  
        # pooling后, shape变成 (1, input_dim)

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.GELU(),
            nn.Linear(hidden_dim * 2, hidden_dim * 2),
            nn.GELU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        # x: (batch_size, 64, 512)
        # 先把 shape (B,64,512) pooling 到 (B,1,512)
        # 这里可以用简单的mean替代，也可以用AdaptiveAvgPool2d
        pooled = x.mean(dim=1)  # (B,512)

        # 送入多层感知机映射到(512)
        out = self.mlp(pooled)  # (B,512)
        return out


In [14]:
ckpt_path = "ckpts/best_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
model = V2AMapperMLP()
model.load_state_dict(torch.load(ckpt_path, map_location="cpu"))
model = model.to(device)

In [5]:
MODEL_NAME = "audioldm-s-full-v2"
audioldm=build_model(model_name=MODEL_NAME)

Load AudioLDM: %s audioldm-s-full-v2
DiffusionWrapper has 185.04 M params.


/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer

In [17]:

class LargeVideoTestDataset(Dataset):
    def __init__(self, data_dir, subset_ratio = 0.2, transform=None):
        """
        root_dir: 保存所有 .pth 文件的目录，每个文件对应一个 sample。
        transform: 如果需要对数据做预处理，可在这里传入。
        """
        super().__init__()
        # 仅收集当前目录下所有的 pth 文件列表
        file_list = []

        for root, dirs, files in os.walk(data_dir):
            for file in files:
                if file.endswith(".pth"):
                    file_list.append(os.path.join(root, file))

        # 仅使用前 20% 的数据
        num_samples = int(len(file_list) * subset_ratio)

        self.file_paths = sorted(file_list)[:num_samples]
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # 在这里按需读取，而不是一次性加载全部
        pth_path = self.file_paths[idx]
        sample_data = torch.load(pth_path)  
        clip_feat = sample_data['clip_features']  # (64, 512)
        clap_feat = sample_data['clap_features']  # (1, 512)
        id = sample_data['id']
        caption = sample_data['caption']

        if self.transform:
            clip_feat, clap_feat = self.transform((clip_feat, clap_feat))

        return clip_feat, clap_feat, id, caption

In [18]:
root = "/mnt/new_volume2/vgg_sound_emb"
partition = "train"
data_dir = f"{root}/{partition}"
vgg_sound = LargeVideoTestDataset(data_dir, subset_ratio = 0.1)

In [19]:
val_ratio = 0.1
test_ratio = 0.1

total_len = len(vgg_sound)
val_len = int(total_len * val_ratio)
test_len = int(total_len * test_ratio)
train_len = total_len - val_len - test_len
train_dataset, val_dataset, test_dataset = random_split(
    vgg_sound, [train_len, val_len, test_len], generator=torch.Generator().manual_seed(42), 
)


In [20]:
test_loader = DataLoader(
    test_dataset, 
    batch_size=1, 
    shuffle=False, 
    num_workers=2,
    pin_memory=True
)

In [21]:

def image2audio(file_name, images, duration=10, guidance_scale=2.5, random_seed=42, n_candidates=3):
    waveform = clap_to_audio(
        latent_diffusion=audioldm,
        clap_feat=images,
        seed=random_seed,
        duration=duration,
        guidance_scale=guidance_scale,
        n_candidate_gen_per_text=int(n_candidates),
    )  # [bs, 1, samples]

    for i, wave in enumerate(waveform):
        # filename = f"{file_name}.wav"
        filename = file_name
        sf.write(filename, wave[0], 16000, 'PCM_16') 

    return waveform

In [22]:
video_path = "/mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video/"

In [ ]:
cnt = 0
for clip_feat, clap_feat, id, caption in tqdm(test_loader):
    # clip_feat: (1, 64, 512)
    # clap_feat: (1, 512)
    # id: (1,)
    # caption: (1,)
    print(clip_feat.shape, clap_feat.shape, id, caption)
    clip_feat = clip_feat.to(device)  # (64, 512)
    # clap_feat = clap_feat.squeeze(0).to(devcice)  # (512)

    gen_clap_feat = model(clip_feat).to(device=device)  # (1, 512)
    
    image2audio(
        file_name="/home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/" + id[0][:-4] + "gen.wav",
        images=gen_clap_feat.to(device=device),
        duration=10,
        guidance_scale=2.5,
        random_seed=42,
        n_candidates=3
    )
    !ffmpeg -i {video_path + id[0]} -vn -acodec pcm_s16le -ar 44100 -ac 2 /home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/{id[0][:-4]}.wav

    cnt += 1
    if cnt == 10:
        break

  0%|          | 0/1826 [00:00<?, ?it/s]

torch.Size([1, 64, 512]) torch.Size([1, 1, 512]) ['-1_f4o5Lqkg_000059.mp4'] ['mosquito buzzing']
batch shape:  [tensor([[ 2.1828e+01,  1.6783e+01,  3.1715e+00,  2.5373e+01,  2.6099e+01,
          2.4544e+01,  1.3202e+01, -1.1837e+00,  2.7100e+00,  3.3163e-01,
          5.0234e+00, -6.6225e+00,  9.7894e+00,  1.6295e+01,  9.4244e+00,
         -6.2355e+00,  2.0865e+00,  5.0002e-01,  3.2720e+01, -8.4438e+00,
          4.0931e+00,  1.8186e+01, -1.1150e+01,  1.0340e+00, -3.7985e+00,
         -2.4221e+01, -4.8938e+01,  2.1103e+00, -1.0755e+01,  3.7562e+01,
          7.1613e+00, -5.1924e+00, -2.9974e+01, -4.7938e+01, -3.8673e+01,
         -7.1633e+00, -1.6234e+00, -1.2147e+01,  2.5153e+01, -8.7885e+00,
          1.3659e+01, -3.0875e+01,  1.6361e+01,  1.8019e+01, -1.4823e+01,
          3.2537e+00, -1.3500e+01, -8.7543e+00, -1.7502e+01, -4.0366e+00,
          5.3313e+00, -2.2422e+01,  8.8823e+00,  2.6822e+00,  2.3186e+01,
         -3.3767e+01, -7.4192e+00,  1.8020e+01, -2.3133e+01,  1.1055e+01,


In [12]:
next_batch = next(iter(test_loader))

In [30]:
clip_feat, clap_feat, id, caption = next_batch

In [31]:
print(clip_feat.shape, clap_feat.shape, id, caption)

torch.Size([1, 64, 512]) torch.Size([1, 1, 512]) ['-1_f4o5Lqkg_000059.mp4'] ['mosquito buzzing']


In [32]:
# ! cp {video_path + id[0]} /home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_video

!ffmpeg -i {video_path + id[0]} -vn -acodec pcm_s16le -ar 44100 -ac 2 /home/ubuntu/project/v2a-mapper/train/test_inf/test_vgg/test_outputs/{id[0]}.wav


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 11.2.0 (Anaconda gcc)
  configuration: --prefix=/home/ubuntu/miniconda3/envs/v2a-mapper --cc=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-cc --ar=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ar --nm=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-nm --ranlib=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ranlib --strip=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libdav1d --enable-zlib --enable-libaom --enable-pic --enable-shared --disable-static --disable-gpl --enable-version3 --disable-sdl2 --enable-libopenh264 --enable-libopus --enable-libmp3lame --enable-l

In [20]:
print(f"clip_feat shape: {clip_feat.shape}")
print(f"clap_feat shape: {clap_feat.shape}")
print(f"id: {id}")
print(f"caption: {caption}")

clip_feat shape: torch.Size([1, 64, 512])
clap_feat shape: torch.Size([1, 1, 512])
id: ['-1_f4o5Lqkg_000059.mp4']
caption: ['mosquito buzzing']


In [24]:
gen_clap_feat = model(clip_feat)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x64 and 512x1024)

In [15]:

MODEL_NAME = "audioldm-s-full-v2"
audioldm=build_model(model_name=MODEL_NAME)

Load AudioLDM: %s audioldm-s-full-v2
DiffusionWrapper has 185.04 M params.


/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense

In [21]:

def image2audio(file_name, images, duration=10, guidance_scale=2.5, random_seed=42, n_candidates=3):
    waveform = clap_to_audio(
        latent_diffusion=audioldm,
        clap_feat=images,
        seed=random_seed,
        duration=duration,
        guidance_scale=guidance_scale,
        n_candidate_gen_per_text=int(n_candidates),
    )  # [bs, 1, samples]

    for i, wave in enumerate(waveform):
        filename = f"{file_name}.wav"
        sf.write(filename, wave[0], 16000, 'PCM_16') 

    return waveform

In [22]:

image2audio(
    file_name="test3",
    images=gen_clap_feat.to(device=devcice),
    duration=10,
    guidance_scale=2.5,
    random_seed=42,
    n_candidates=3
)

batch shape:  [tensor([[  8.4076,   8.9150,   8.1998,  20.5605,  15.6725,  16.5475,   5.3757,
          -3.4283,  -0.2870,   3.5352,  -0.9050, -10.0235,   7.3880,   9.9746,
           7.2894,  -5.4092,   2.0637,   0.5552,  20.3179,  -1.8790,   5.2214,
           8.8170, -10.7283,   2.8796,  -4.3717, -20.2293, -34.6853,   0.9166,
          -8.3378,  28.2999,   5.3379,  -3.9145, -16.4478, -31.0907, -21.6145,
          -2.3070,  -3.5866, -10.9111,  15.1106,  -7.7051,   8.4162, -21.8627,
           7.8714,  11.9345, -11.4393,   3.4052,  -5.2862,  -5.9282, -10.0566,
          -6.0752,   1.0733, -12.8453,   2.8973,  -1.8153,  13.1791, -19.0822,
          -6.3574,   6.2898, -11.7146,   8.4148,   7.1210,   9.3209,  10.1412,
          27.7904,  -1.6135,   4.5770,  -2.2091,   5.5536,   2.7890,   4.5220,
           5.8580,  11.6451,  -3.7875,  -5.1803,  25.6695, -10.0351, -19.0158,
          -3.7769,  -0.2103,  10.7586,  -3.6818,  -2.6625,   4.5504,  25.3820,
         -12.0752,  38.3779, -13.6586

DDIM Sampler: 100%|██████████| 200/200 [00:17<00:00, 11.56it/s]
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


array([[[ 6.1683670e-05,  2.8324412e-04,  8.3505845e-05, ...,
         -4.0632939e-01, -3.9695010e-01, -4.3544635e-01]]], dtype=float32)